## Dist2Inlet testing

The second shoreline is producing values of 671.67... for dist2inlet at every transect. What is going on?!

In [1]:
import os
import sys
import time
import shutil
import pandas as pd
import numpy as np
import arcpy
import CoastalVarExtractor.functions_warcpy as fwa
import CoastalVarExtractor.functions as fun

from CoastalVarExtractor.setvars import *

import matplotlib.pyplot as plt
import matplotlib

site: Fisherman
year: 2014
setvars.py initialized variables.


In [2]:
shoreline
in_trans = extendedTrans

In [123]:
utmSR = arcpy.Describe(in_trans).spatialReference
df = pd.DataFrame(columns=[tID_fld, 'Dist2Inlet']) # initialize dataframe

In [124]:
# Get inlets geometry objects
inlets = [row[0] for row in arcpy.da.SearchCursor(inletLines, ("SHAPE@"))]
len(inlets)

4

In [125]:
# there is only one shoreline feature because it is dissolved. Is this an issue?
lines = [row[0] for row in arcpy.da.SearchCursor(shoreline, ("SHAPE@"))]
line = lines[0]
len(lines)

1

In [126]:
tcur = arcpy.da.SearchCursor(in_trans, ("SHAPE@",  tID_fld))

### Start loop

In [127]:
[transect, tID] = tcur.next()
print(tID)
line.disjoint(transect)
mdist = 0

1


In [132]:
while tID < 194:
    [transect, tID] = tcur.next()
    print('\n')
    line.disjoint(transect)
    # 1. cut shoreline at the transect
    print(tID)
    [rseg, lseg] = line.cut(transect)
    # print(rseg.length)
    # print(lseg.length)
    # 2. if the shoreline segment touches any inlet, get the segment length.
    # in case of multipart features, use only the first part
    # What happens when the shoreline and transect intersect in multiple places?
    lenR = arcpy.Polyline(rseg.getPart(0), utmSR).length if not all(rseg.disjoint(i) for i in inlets) else np.nan
    lenL = arcpy.Polyline(lseg.getPart(0), utmSR).length if not all(lseg.disjoint(i) for i in inlets) else np.nan        
    print([rseg.length, lenR])
    print([lseg.length, lenL])
    # 3. Return the length of the shorter segment and save it in the DF
    mindist = np.nanmin([lenR, lenL])
    if mdist == mindist:
        print('<<<CONSECUTIVE dist2inlet VALUES ARE THE SAME at tID {}>>>'.format(tID))
    mdist = mindist
    print({tID_fld:tID, 'Dist2Inlet':mindist})
    df = df.append({tID_fld:tID, 'Dist2Inlet':mindist}, ignore_index=True)

# Starting at tID 19, lenL was always 550.72... Replicated twice
# After tID 12, lenL is always 550.72.
# REASON!! the first part is always the first shoreline segment, not one side of where it's split.
# Option A: convert the shoreline feature to single part - is there a reason this wasn't done before?
# Option B: add a condition that the part touch the transect that was used to cut it.
# This problem is unique because the shorelines originated from the same boundary polygon. CreateShoreBetweenInlets dissolves by the FID of the boundary polygons.
# Add a step to CreateShore... that converts multipart to singlepart or make that requirement.

# If we used the rseg.length and lseg.length instead, this would not be a problem. 
# I don't know whether those are accurate or if they might be multipart that don't intersect with an inlet.



2
[125.59829834899217, 125.59829834899217]
[9693.468379838814, 425.12255506460576]
{'Dist2Inlet': 125.59829834899217, 'sort_ID': 2}


3
[175.06353683321097, 175.06353683321097]
[9644.003141354595, 375.657316580387]
{'Dist2Inlet': 175.06353683321097, 'sort_ID': 3}


4
[215.22030323512914, 215.22030323512914]
[9603.846374952678, 335.50055017846876]
{'Dist2Inlet': 215.22030323512914, 'sort_ID': 4}


5
[261.77246393428044, 261.77246393428044]
[9557.294214253527, 288.9483894793175]
{'Dist2Inlet': 261.77246393428044, 'sort_ID': 5}


6
[306.18960954840037, 306.18960954840037]
[9512.877068639405, 244.53124386519752]
{'Dist2Inlet': 244.53124386519752, 'sort_ID': 6}


7
[352.1535377765194, 352.1535377765194]
[9466.913140411287, 198.56731563707848]
{'Dist2Inlet': 198.56731563707848, 'sort_ID': 7}


8
[403.2052774051603, 403.2052774051603]
[9415.861400782645, 147.51557600843762]
{'Dist2Inlet': 147.51557600843762, 'sort_ID': 8}


9
[450.91936414701166, 450.91936414701166]
[9368.147314040794, 99.8

In [128]:
df

,sort_ID,Dist2Inlet


In [129]:
# 1. cut shoreline at the transect
print(tID)
[rseg, lseg] = line.cut(transect)
print(rseg.length)
print(lseg.length)

1
79.01685813656094
9740.049820051247


In [130]:
# 2. if the shoreline segment touches any inlet, get the segment length.
# in case of multipart features, use only the first part
# What happens when the shoreline and transect intersect in multiple places?
lenR = arcpy.Polyline(rseg.getPart(0), utmSR).length if not all(rseg.disjoint(i) for i in inlets) else np.nan
lenL = arcpy.Polyline(lseg.getPart(0), utmSR).length if not all(lseg.disjoint(i) for i in inlets) else np.nan        
print(lenR)
print(lenL)

79.01685813656094
471.70399527703694


In [131]:
# 3. Return the length of the shorter segment and save it in the DF
mindist = np.nanmin([lenR, lenL])
print({tID_fld:tID, 'Dist2Inlet':mindist})
df = df.append({tID_fld:tID, 'Dist2Inlet':mindist}, ignore_index=True)
df

{'Dist2Inlet': 79.016858136560941, 'sort_ID': 1}


,sort_ID,Dist2Inlet
0,1.0,79.016858


### End loop

In [25]:
df.index = df[tID_fld]
df.drop(tID_fld, axis=1, inplace=True)

In [26]:
df

,Dist2Inlet
sort_ID,
NaN,79.016858
2.0,125.598298
